# Linear Regression



Modelo de Regressão Linear para predição de SCORE.


In [1]:
%run ../src/feature_engineering.py
%run ../src/model_evaluation.py
%run ../src/plot.py
%run ../src/pipeline.py

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
#from sklearn.cross_validation import cross_val_score, ShuffleSplit

%matplotlib inline
#Alterando configurações padrão dos plots
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20, 60)



In [5]:
course_df = pd.read_csv('../data/preprocessed/edx_2014_train.csv',low_memory=False)

class LR:
    def provide(self):
        return LinearRegression()
    
course_df['grade'] = course_df['grade'].map(lambda g: g if g > 0 else 0)
    
results, plt = train_evaluate([LR()], course_df, split_criteria='course_id', feature_extractor=feature_extract, 
                              label='grade', evaluation=regression_evaluation, 
                              plots= [scatter_real_vs_pred_subplot, scatter_residual_error_subplot])

plt.show()



2019-05-31 08:15:12 Iniciando modelo LR
2019-05-31 08:15:12 Iniciando curso CB22x


AttributeError: 'LR' object has no attribute 'sample_size'

In [10]:
reform = {(level2_key, level3_key): values
              for level1_key, level2_dict in results.items()
              for level2_key, level3_dict in level2_dict.items()
              for level3_key, values      in level3_dict.items()}

In [15]:
pd.options.display.float_format = '{:,.4f}'.format
pd.DataFrame(reform, index=['LinearRegression']).T

LinearRegression
2.01x  explained_variance_score            0.9019
       mean_absolute_error                 0.0240
       mean_squared_error                  0.0032
       r2_score                            0.9019
6.002x explained_variance_score            0.9360
       mean_absolute_error                 0.0174
       mean_squared_error                  0.0022
       r2_score                            0.9360
6.00x  explained_variance_score            0.9012
       mean_absolute_error                 0.0147
       mean_squared_error                  0.0015
       r2_score                            0.9012
7.00x  explained_variance_score            0.9162
       mean_absolute_error                 0.0179
       mean_squared_error                  0.0021
       r2_score                            0.9162
8.02x  explained_variance_score            0.9324
       mean_absolute_error                 0.0123
       mean_squared_error                  0.0013
       r2_score                            0.9323
8.MReV explained_variance_score            0.9191
       mean_absolute_error                 0.0160
       mean_squared_error                  0.0016
       r2_score                            0.9191
CB22x  explained_variance_score            0.9052
       mean_absolute_error                 0.0076
       mean_squared_error                  0.0007
       r2_score                            0.9052
CS50x  explained_variance_score            1.0000
       mean_absolute_error                 0.0000
       mean_squared_error                  0.0000
       r2_score                            1.0000
ER22x  explained_variance_score            0.9288
       mean_absolute_error                 0.0140
       mean_squared_error                  0.0020
       r2_score                            0.9288
PH207x explained_variance_score            0.9013
       mean_absolute_error                 0.0235
       mean_squared_error                  0.0042
       r2_score                            0.9013

### Um Modelo para Todos os Cursos

In [3]:
course_df = pd.read_csv('../data/preprocessed/edx_2014_train.csv',low_memory=False)


In [49]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import pearsonr

class ModelDecorator:
    def __init__(self, model):
        self._model = model
        
    def fit(self, X, y):
        return self._model.fit(X, y, epochs=100, verbose=0)
    
    def predict(self, X):
        flatten = lambda l: [item for sublist in l for item in sublist]
        return flatten(self._model.predict(X))

class _LinearRegression:
    def provide(self):
        return LinearRegression()
    def sample_size(self):
        return None
    
class _DecisionTree:
    def provide(self):
        return DecisionTreeRegressor(criterion='mae', max_depth=3)
    def sample_size(self):
        return 20000
    
class _DeepLearning:
    def provide(self):
        model = models.Sequential()
        model.add(layers.Dense(13, activation='relu', input_dim=13))
        model.add(layers.Dense(13, activation='relu'))    
        model.add(layers.Dense(1))
        model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
        return ModelDecorator(model)
    def sample_size(self):
        return None
    
class _SupportVectorMachine:
    def provide(self):
        return Pipeline([('scaler', StandardScaler()),
                  ('poly', SVR(kernel='poly', C=1e3, cache_size=7000))])
    def sample_size(self):
        return 10000
        
class _RandonForest:
    def provide(self):
        return Pipeline([('scaler', StandardScaler()),
                  ('rfr', RandomForestRegressor(criterion='mae'))])
    def sample_size(self):
        return 20000

_models = [
    _LinearRegression(),
#    _DecisionTree(),
#    _DeepLearning(),
#    _SupportVectorMachine(),
#    _RandonForest()
]
    
model  = _LinearRegression()

data = feature_extract(course_df)
X0, Y = split_xy(data)


std = StandardScaler()
X = pd.DataFrame(std.fit_transform(X0))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

scores = []
for i in range(X_train.shape[1]):     
    model= _LinearRegression().provide()
    model.fit(X_train[X.columns[i]].reshape(-1, 1), y_train)
    y_pred = model.predict(X_test[X.columns[i]].reshape(-1, 1))
    scores.append((pearsonr(y_test, y_pred)[0][0], X0.columns[i]))

pd.DataFrame(sorted(scores))

feats = ['grade']
for score in scores:
    value, feat = score
    if value >= 0.1:
        feats.append(feat)

feats        
        
def feat_extract_selected(df):
    data = feature_extract(df)
    return data[feats]
    
course_df['grade'] = course_df['grade'].map(lambda g: g if g > 0 else 0)
    
results, plt = train_evaluate_by_course(_models, course_df, feature_extractor=feat_extract_selected, 
                              label='grade', split_criteria='course_id', evaluation=regression_evaluation, 
                              plots= [scatter_real_vs_pred_subplot, scatter_residual_error_subplot])


D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:75: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
D:\Programs\Anaconda\lib\site-packages\ipykernel_launcher.py:76: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


['grade',
 'viewed',
 'explored',
 'certified',
 'nevents',
 'ndays_act',
 'nplay_video',
 'nchapters',
 'nforum_posts']

In [47]:
reform = {(level2_key, level3_key): values
              for level1_key, level2_dict in results.items()
              for level2_key, level3_dict in level2_dict.items()
              for level3_key, values      in level3_dict.items()}
pd.options.display.float_format = '{:,.4f}'.format
pd.DataFrame(reform, index=['LinearRegression']).T

LinearRegression
2.01x  explained_variance_score            0.8891
       mean_absolute_error                 0.0215
       mean_squared_error                  0.0036
       r2_score                            0.8888
6.002x explained_variance_score            0.9287
       mean_absolute_error                 0.0165
       mean_squared_error                  0.0026
       r2_score                            0.9281
6.00x  explained_variance_score            0.8493
       mean_absolute_error                 0.0154
       mean_squared_error                  0.0022
       r2_score                            0.8493
7.00x  explained_variance_score            0.8752
       mean_absolute_error                 0.0184
       mean_squared_error                  0.0030
       r2_score                            0.8752
8.02x  explained_variance_score            0.9126
       mean_absolute_error                 0.0139
       mean_squared_error                  0.0017
       r2_score                            0.9114
8.MReV explained_variance_score            0.8489
       mean_absolute_error                 0.0193
       mean_squared_error                  0.0034
       r2_score                            0.8489
CB22x  explained_variance_score            0.8897
       mean_absolute_error                 0.0084
       mean_squared_error                  0.0009
       r2_score                            0.8892
CS50x  explained_variance_score            0.9216
       mean_absolute_error                 0.0071
       mean_squared_error                  0.0006
       r2_score                            0.9209
ER22x  explained_variance_score            0.9207
       mean_absolute_error                 0.0148
       mean_squared_error                  0.0022
       r2_score                            0.9207
PH207x explained_variance_score            0.8684
       mean_absolute_error                 0.0228
       mean_squared_error                  0.0056
       r2_score                            0.8642

In [48]:
result2 = []
for modelname, courses in results.items():
    for course_id, metrics in courses.items():
        result2.append({'model': modelname, 'course_id': course_id, 'mae':metrics['mean_absolute_error']})
    
pd.options.display.float_format = '{:,.4f}'.format
pd.DataFrame(result2)

,course_id,mae,model
0,CB22x,0.0084,_LinearRegression
1,CS50x,0.0071,_LinearRegression
2,ER22x,0.0148,_LinearRegression
3,PH207x,0.0228,_LinearRegression
4,6.002x,0.0165,_LinearRegression
5,2.01x,0.0215,_LinearRegression
6,6.00x,0.0154,_LinearRegression
7,7.00x,0.0184,_LinearRegression
8,8.02x,0.0139,_LinearRegression
9,8.MReV,0.0193,_LinearRegression
